Google News Sentiment Analysis

In [ ]:
%config IPCompleter.greedy=True

In [ ]:
pip install flair

     |████████████████████████████████| 284 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 981 kB 1.8 MB/s eta 0:00:01
     |████████████████████████████████| 19.7 MB 2.3 MB/s eta 0:00:01
     |████████████████████████████████| 2.0 MB 2.4 MB/s eta 0:00:01
     |████████████████████████████████| 66 kB 2.8 MB/s eta 0:00:01
     |████████████████████████████████| 108.9 MB 210 kB/s eta 0:00:011
     |████████████████████████████████| 1.1 MB 4.3 MB/s eta 0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 965 kB 3.9 MB/s eta 0:00:01
     |████████████████████████████████| 24.2 MB 512 kB/s eta 0:00:01
     |████████████████████████████████| 788 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 2.2 MB 790 kB/s eta 0:00:01
     |████████████████████████████████| 883 kB 4.9 MB/s eta 0:00:01
     |████████████████████████████████| 119 kB 4.2 MB/

In [ ]:
pip install textblob

     |████████████████████████████████| 636 kB 1.6 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
import flair
from textblob import TextBlob
import os
import datetime
import numpy as np
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [ ]:
dateFormat = '%Y-%m-%d'

In [ ]:
def performSentimentAnalysis(inputFilename, outputFileName):
    """
    :param news_df: panda dataframe, with index as date of the format dateFormat
    :return: another dataframe with same index as news_df dataframe and additional columns as sentiment analysis 
    values
    """
    newsDf = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/' + inputFilename, index_col=0)
    #del df['Unnamed: 0']
    #del df['news_9_url']
    #del df['news_10_url']
    del newsDf['news_9_text']
    del newsDf['news_10_text']
    newsColumns = newsDf.columns
    
    """
    Text classification is a supervised machine learning method used to classify sentences or text documents into
    one or more defined categories. It’s a widely used natural language processing task playing an important role
    in spam filtering, sentiment analysis, categorisation of news articles and many other business related issues.
    Most current state of the art approaches rely on a technique called text embedding. It transforms text into a
    numerical representation in high-dimensional space. It allows for a document, sentence, word, a character 
    (depending on what embedding we use) to be expressed as a vector in this high-dimensional space.
    """
    
    #loads pretrained text classifier model
    flairSentiment = flair.models.TextClassifier.load('en-sentiment')
    sentiAnalyze = SentimentIntensityAnalyzer()
    
    """
    The new release 0.4 comes with two pre-trained models. A sentiment analysis model trained on the IMDB dataset
    and an ‘offensive language detection’ model (which currently only supports German).
    Using, downloading and storing the model has all been incorporated into a single method that makes the whole
    process of using pre-trained models surprisingly straightforward.
    """
        
    tVar = 0
    
    #iterating through each of the rows in news_df
    for rowIndex, row in newsDf.iterrows():
        tVar += 1
        print(rowIndex)
        
        #initializing required dictionaries and variables
        sentimentCount = 0
        sentimentPolarityDict = dict()
        sentimentSubjectivityDict = dict()
        flairSentimentDict = dict()

        positiveSentDict = dict()
        negativeSentDict = dict()
        neutralSentDict = dict()
        compoundSentDict = dict()

        flairSentimentCount = 0

        polarity = 0
        subjectivity = 0

        totalPosScore = 0
        totalNegScore = 0
        totalNeuScore = 0
        totalComScore = 0

        # sid_polarity_total = {'neg': 0., 'neu': 0., 'pos': 0., 'compound': 0.}

        for col in range(len(newsColumns)):
            data = (str(row[col]))
            print('\t', col)
            if data == 'NaN':
                continue
            
            """
            The sentiment property returns a named tuple of the form Sentiment(polarity, subjectivity). 
            The polarity score is a float within the range [-1.0, 1.0]. The subjectivity is a float within 
            the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
            """
            polarity += TextBlob(data).sentiment[0]
            subjectivity += TextBlob(data).sentiment[1]
            
            #provides us with number of tokens in the sentence along with their ids
            flairSentenceTokenizer = flair.data.Sentence(data)
            
            #provides us with the polarity score for the given sentence alongwith the polarity type i.e. negative /
            #positive / neutral
            flairSentiment.predict(flairSentenceTokenizer)
            flairLabelsCount = flairSentenceTokenizer.labels
            
            flairFloatValue = getSentimentValue(flairLabelsCount)
            flairSentimentCount += flairFloatValue
            
            #provides us with the breakdown of the polarity score as negative/neutral/positive/compound
            #{'neg': 0.039, 'neu': 0.809, 'pos': 0.152, 'compound': 0.9914
            polarityBreakdown = sentiAnalyze.polarity_scores(data)
            totalPosScore += polarityBreakdown['pos']
            totalNegScore += polarityBreakdown['neg']
            totalNeuScore += polarityBreakdown['neu']
            totalComScore += polarityBreakdown['compound']

            sentimentCount += 1

        print(str(rowIndex), ' ', tVar)
        
        flairSentimentDict[str(rowIndex)] = flairSentimentCount / sentimentCount
        
        sentimentPolarityDict[str(rowIndex)] = polarity / sentimentCount
        
        sentimentSubjectivityDict[str(rowIndex)] = subjectivity / sentimentCount
        
        print(flairSentimentDict[str(rowIndex)], sentimentPolarityDict[str(rowIndex)],
              sentimentSubjectivityDict[str(rowIndex)])
        
        #Flair
        flairDf = pd.DataFrame.from_dict(flairSentimentDict, orient='index', columns=['googleNewsFlair'])
        flairDf.index.name = 'date'
        
        #Polarity
        polarityDf = pd.DataFrame.from_dict(sentimentPolarityDict, orient='index',columns=['googleNewsPolarity'])
        polarityDf.index.name = 'date'
        
        #Subjectivity
        subjectivityDf = pd.DataFrame.from_dict(sentimentSubjectivityDict, orient='index',columns=['googleNewsSubjectivity'])
        subjectivityDf.index.name = 'date'

        positiveSentDict[str(rowIndex)] = totalPosScore / sentimentCount
        negativeSentDict[str(rowIndex)] = totalNegScore / sentimentCount
        neutralSentDict[str(rowIndex)] = totalNeuScore / sentimentCount
        compoundSentDict[str(rowIndex)] = totalComScore / sentimentCount

        positiveDf = pd.DataFrame.from_dict(positiveSentDict, orient='index',columns=['googleNewsPositive'])
        positiveDf.index.name = 'timestamp'

        negativeDf = pd.DataFrame.from_dict(negativeSentDict, orient='index',columns=['googleNewsNegative'])
        negativeDf.index.name = 'timestamp'

        neutralDf = pd.DataFrame.from_dict(neutralSentDict, orient='index',columns=['googleNewsNeutral'])
        neutralDf.index.name = 'timestamp'

        compoundDf = pd.DataFrame.from_dict(compoundSentDict, orient='index',columns=['googleNewsCompound'])
        compoundDf.index.name = 'timestamp'
        
        #concatenating all the dataframes into a single dataframes
        mergeDf = pd.concat([flairDf, polarityDf, subjectivityDf, positiveDf, negativeDf, neutralDf, compoundDf], axis=1)

        # if file does not exist write header 
        if not os.path.isfile('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/Sentiment Analysis/' + outputFileName):
            mergeDf.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/Sentiment Analysis/' + outputFileName)
        else: # else it exists so append without writing the header
            mergeDf.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/Sentiment Analysis/' + outputFileName, mode='a', header=False)

    return


In [ ]:
def getSentimentValue(sentimentPolarity):
    """
    parse input of the format [NEGATIVE (0.9284018874168396)] and return +ve or -ve float value
    :param sentiments:
    :return:
    """
    sentimentStr = str(sentimentPolarity)
    negative = 'NEGATIVE' in sentimentStr
    if negative:
        sentimentStr = sentimentStr.replace('NEGATIVE', '')
    else:
        sentimentStr = sentimentStr.replace('POSITIVE', '')

    sentimentStr = sentimentStr.replace('(', '').replace('[', '').replace(')', '').replace(']', '')

    value = float(sentimentStr)
    if negative:
        return -value
    return value

In [ ]:
def processSentimentAnalysis(inputFilename, outputFilename):
    # drop duplicates and sort
    processeddf = pd.read_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/Sentiment Analysis/' + inputFilename, index_col=0)
    processeddf.index = pd.to_datetime(processeddf.index)
    uniqueIndex = np.unique(processeddf.index, return_index=True)[1]
    processeddf = processeddf.iloc[uniqueIndex]
    processeddf.to_csv('/Volumes/Sashank Work/MSc DCU/Semester 2/CA683 Data Analytics and Data Mining/Assignment/Dataset/Google News/Sentiment Analysis/' + outputFilename)

In [ ]:
if __name__ == "__main__":
    inputFilename = 'google_news_final_cleaned.csv'
    outputFilename = inputFilename[0:-4] + '_sentiment.csv'
    performSentimentAnalysis(inputFilename, outputFilename)
    processSentimentAnalysis(outputFilename, outputFilename)

2021-03-23 16:58:54,922 loading file /Users/hitman94/.flair/models/sentiment-en-mix-distillbert_4.pt
2018-01-01
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-01   1
-0.196575 0.16857131696996558 0.45076064813350114
2018-01-02
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-02   2
-0.7423249999999999 0.10340684171236708 0.37445111582425894
2018-01-03
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-03   3
-0.566125 0.12508625533614132 0.37784867164992114
2018-01-04
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-04   4
-0.5702375 0.11171615331649404 0.3787623869345118
2018-01-05
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-05   5
-0.316825 0.09452768375653221 0.40858606006056053
2018-01-06
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-06   6
-0.467475 0.09368699964916644 0.340521282083152
2018-01-07
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-07   7
-0.4971499999999999 0.1279523066670707 0.43949024519024604
2018-01-08
	 0
	 1
	 2
	 3
	 4
	 5
	 6
	 7
2018-01-08   8
-0.49829999999999997 0.15209225598612047 0.4313594096275698
2018-01-